# Poison Certified Training on UCI Datasets

In [1]:
%load_ext autoreload
%autoreload 2
import torch
import abstract_gradient_training as agt
from models.fully_connected import FullyConnected 
from datasets import uci

In [2]:
# configure the training parameters
batchsize = 20000
config = agt.AGTConfig(
    fragsize=20000,
    learning_rate=0.005,
    epsilon=0.01,
    k_poison=200,
    n_epochs=1,
    device="cuda:1",
    forward_bound="interval",
    backward_bound="interval",
    loss="mse",
)
torch.manual_seed(0)

In [3]:
# initialize the model and dataset
dl_train, dl_test = uci.get_dataloaders(batchsize, batchsize, "houseelectric")
model = FullyConnected(11, 1, 64, 1)  # network with 1 hidden layer of 64 neurons

houseelectric dataset, N=2049280, d=11


In [4]:
# train the model
param_l, param_n, param_u = agt.poison_certified_training(model, config, dl_train, dl_test)

Training Batch: 92it [00:35,  2.56it/s, Bound: 0.00408 Network eval: Worst=0.042, Nominal=0.038, Best=0.034] 


In [5]:
# evaluate the trained model
mse = agt.test_metrics.test_mse(param_n, param_l, param_u, dl_test)
print(f"Test MSE: nominal = {mse[1]:.4g}, certified upper bound = {mse[0]:.4g}, certified lower bound = {mse[2]:.4g}")

Test MSE: nominal = 0.03772, certified upper bound = 0.04172, certified lower bound = 0.03398
